In [1]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import requests
from bs4 import BeautifulSoup

def extract_info_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        info_text = '\n'.join(p.text for p in paragraphs)
        return info_text
    else:
        print("Failed to retrieve data from URL:", url)
        return None

def save_to_text_file(text, filename, folder='extracted_info'):
    if not os.path.exists(folder):
        os.makedirs(folder)
    file_path = os.path.join(folder, filename)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)

if __name__ == "__main__":
    list_of_URLS = [
        'https://catalogue.uci.edu/allcourses/compsci/',
        'https://catalogue.uci.edu/donaldbrenschoolofinformationandcomputersciences/departmentofcomputerscience/computerscience_bs/#sampleprogramtext',
        'https://catalogue.uci.edu/donaldbrenschoolofinformationandcomputersciences/departmentofcomputerscience/computerscience_bs/#requirementstext'
    ]
    for url in list_of_URLS: 
        info_text = extract_info_from_url(url)
        print("Extracted info from {}".format(url))
        if info_text:
            file_name = 'extracted_info_{}.txt'.format(url.split('/')[-2])
            save_to_text_file(info_text, file_name)
            print("Information extracted and saved to '{}'".format(file_name))


Extracted info from https://catalogue.uci.edu/allcourses/compsci/
Information extracted and saved to 'extracted_info_compsci.txt'
Extracted info from https://catalogue.uci.edu/donaldbrenschoolofinformationandcomputersciences/departmentofcomputerscience/computerscience_bs/#sampleprogramtext
Information extracted and saved to 'extracted_info_computerscience_bs.txt'
Extracted info from https://catalogue.uci.edu/donaldbrenschoolofinformationandcomputersciences/departmentofcomputerscience/computerscience_bs/#requirementstext
Information extracted and saved to 'extracted_info_computerscience_bs.txt'
